In [1]:
import numpy as np
import matplotlib.pyplot as plt

# BCD SVM

In [47]:
class SVM:
    def __init__(self,kernel='linear',C=1,**kernel_coef):
        self.kernel=kernel
        self.kernel_coef=kernel_coef
        self.C=C

    def kernelFun(self,x1,x2,method):
        d=self.kernel_coef.get('d')
        beta=self.kernel_coef.get('beta')
        theta=self.kernel_coef.get('theta')
        sigma=self.kernel_coef.get('sigma')
        if method=='linear':
            return x1@x2
        elif method=='Polynomial':
            return (x1@x2)**d
        elif method=='Sigmoid':
            return np.tanh(beta*(x1@x2)+theta)
        elif method=='Gauss':
                return np.exp(-np.linalg.norm(x1-x2)**2/(2*sigma**2))
        elif method=='Laplace': 
            return np.exp(-np.linalg.norm(x1-x2)/sigma)
    
    def fit(self,X,y,iterations=2000):
        m=y.size
        if X.shape[0]==m:       # Standardize shape
            X=X.T
        if len(y.shape)>1:
            y.reshape(m)
        self.Xtrain=X
        self.ytrain=y
        
        K=np.empty((m,m))       # Formulate K Matrix
        if callable(self.kernel):
            for i in range(m):
                for j in range(m):
                    K[i,j]=self.kernel(X[:,i],X[:,j])
        else:
            for i in range(m):
                for j in range(m):
                    K[i,j]=self.kernelFun(X[:,i],X[:,j],method=self.kernel)

        self.b=0
        self._lambda=np.zeros(m)
        for iteration in range(iterations):
            f=np.apply_along_axis(self.eval,axis=0,arr=X)
            con1=(self._lambda==0)
            con2=(self._lambda==self.C)
            err=y*f-1
            err[(con1 & (err>=0))|(con2 & (err<=0))|((~con1&~con2)&(err==0))]=0
            if  (err==0).all():
                break
            i=(err**2).argmax()
            temp=(f-f[i]).copy()
            temp[i]=-np.inf
            j=temp.argmax()

            t=0
            for k in range(m):
                if k==i or k==j:
                    continue
                else:
                    t-=self._lambda[k]*y[k]

            if y[i]==y[j]:
                H=min(self.C,self._lambda[i]+self._lambda[j])               # Higher Bound
                L=max(0,self._lambda[i]+self._lambda[j]-self.C)             # Lower Bound
            else:
                H=min(self.C,self.C+self._lambda[i]-self._lambda[j])
                L=max(0,self._lambda[i]-self._lambda[j])

            # if y[i]==y[j]:
            #     H=min(self.C,self.C-y[j]*t) # Higher Bound
            #     L=max(0,-y[j]*t)            # Lower Bound
            # else:
            #     H=min(self.C,y[j]*t)
            #     L=max(0,y[j]*t-self.C)

            _lambda_i_hat=self._lambda[i]+\
                y[i]*((f[j]-y[j])-(f[i]-y[i]))/(K[i,i]+K[j,j]-2*K[i,j])
            if _lambda_i_hat>H:             # Project lambda_i
                self._lambda[i]=H
            elif _lambda_i_hat<L:
                self._lambda[i]=L
            else:
                self._lambda[i]=_lambda_i_hat
            self._lambda[j]=y[j]*(t-self._lambda[i]*y[i])

            b_i=y[i]                        # Update b
            for k in range(m):
                b_i-=self._lambda[k]*y[k]*K[i,k]
            b_j=y[j]                        # Update b
            for k in range(m):
                b_j-=self._lambda[k]*y[k]*K[j,k]
            self.b=(b_i+b_j)/2

        if self.kernel=='linear':
            self.w=np.zeros(X.shape[0])
            for i in range(m):
                self.w+=self._lambda[i]*y[i]*X[:,i]
    

    def eval(self, x):
        y_pred=self.b
        if callable(self.kernel):
            for i in range(self.ytrain.size):
                y_pred+=self._lambda[i]*self.ytrain[i]*self.kernel(self.Xtrain[:,i],x)
        elif self.kernel=='linear' and hasattr(self,'w'):   # Simplify calculation
                y_pred+=self.w@x
        else:
            for i in range(self.ytrain.size):
                y_pred+=self._lambda[i]*self.ytrain[i]*\
                    self.kernelFun(self.Xtrain[:,i],x,method=self.kernel)
        return y_pred
    
    def predict(self,x):
        reg=self.eval(x)
        if reg<0:
            return -1
        else:
            return 1


# ADMM SVM

In [38]:
class ADMMsvm(SVM):
    def fit(self,X,y,rho=0.5,iterations=5000):
        self.rho=rho
        m=y.size
        if X.shape[0]==m:
            X=X.T
        if len(y.shape)>1:
            y.reshape(m)
        self.Xtrain=X
        self.ytrain=y

        K=np.empty((m,m))
        if callable(self.kernel):
            for i in range(m):
                for j in range(m):
                    K[i,j]=self.kernel(X[:,i],X[:,j])
        else:
            for i in range(m):
                for j in range(m):
                    K[i,j]=self.kernelFun(X[:,i],X[:,j],method=self.kernel)

        Y=np.diag(y)
        _lambda=np.zeros(m)
        _mu=np.zeros(m+1)
        _z=Y@_lambda

        augI=np.eye(m,m+1)
        augY=np.column_stack((Y,y))
        invM4z=np.linalg.inv(K+rho+np.eye(m))
        invM4lmbda=np.linalg.inv(Y**2+np.outer(y,y))
        oneDrho=np.ones(m)/rho
        Gamma1=np.row_stack((Y,y))
        Gamma2=np.eye(m+1,m)
        for iteration in range(iterations):
            _z=rho*invM4z@(Y@_lambda+augI@_mu)
            _lambda_hat=invM4lmbda@(oneDrho+Y@_z-augY@_mu)
            _lambda=_lambda_hat
            _lambda[_lambda_hat<0]=0
            _lambda[_lambda_hat>self.C]=self.C
            _mu=_mu+Gamma1@_lambda-Gamma2@_z
        self._z=_z
        self._lambda=_lambda
        self._mu=_mu

        is_support_vector=(_lambda<self.C) & (_lambda>0)
        if self.kernel=='linear':
            self.w=np.zeros(X.shape[0])
            for i in range(m):
                self.w+=_lambda[i]*y[i]*X[:,i]
            self.b=y[is_support_vector].mean()-\
                self.w@X[:,is_support_vector].mean(axis=1)
        else:
            self.b=0
            for i in range(m):
                if is_support_vector[i]:
                    temp=y[i]
                    for j in range(m):
                        temp-=_lambda[i]*y[i]*K[i,j]
                self.b+=temp
            self.b=self.b/is_support_vector.sum()

# test

## data

In [4]:
# if __name__=='__main__':
n=100
np.random.seed(42)

#以随机生成分界线，在两侧随机生成数据
random1=np.random.random(2)#随机分界线的法向量
random2=20*np.random.random(n)-10#分界线两侧数据的偏移量
x1=20*np.random.random(n)-10
x2=x1*random1[0]/random1[1]+random2
x=np.array(list(zip(x1,x2)))
y=np.zeros(n)
y[random2>0]=1#设分界线上方的点标签为1
y[random2<0]=-1#设分界线下方的点标签为-1

## BCD SVM

In [48]:
svm=SVM()
svm.fit(x,y)
y_pred=np.empty(n)
for i in range(n):
    y_pred[i]=svm.predict(x[i,:])

## ADMM SVM

In [39]:
Asvm=ADMMsvm()
Asvm.fit(x,y)
Ay_pred=np.empty(n)
for i in range(n):
    Ay_pred[i]=Asvm.predict(x[i,:])

## sklearn

In [6]:
from sklearn.svm import SVC
skSVM=SVC()
skSVM.fit(x,y)
sky_pred=np.empty(n)
for i in range(n):
    sky_pred[i]=skSVM.predict(x[[i],:])

## Comparision

In [46]:
(y!=y_pred).sum()

2

In [40]:
(y!=Ay_pred).sum()

0

In [8]:
(y!=sky_pred).sum()

0